In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls '/content/drive/My Drive/schubert'

schu_143_1.mid	  schub_d960_2.mid     schubert_D935_2.mid  schumm-2.mid
schu_143_2.mid	  schub_d960_3.mid     schubert_D935_3.mid  schumm-3.mid
schu_143_3.mid	  schub_d960_4.mid     schubert_D935_4.mid  schumm-4.mid
schub_d760_1.mid  schubert_D850_1.mid  schuim-1.mid	    schumm-5.mid
schub_d760_2.mid  schubert_D850_2.mid  schuim-2.mid	    schumm-6.mid
schub_d760_3.mid  schubert_D850_3.mid  schuim-3.mid
schub_d760_4.mid  schubert_D850_4.mid  schuim-4.mid
schub_d960_1.mid  schubert_D935_1.mid  schumm-1.mid


# **WaveNet**
![alt text](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/12/arcg.jpg)

**The Workflow of WaveNet:**


*   Input is fed into a causal 1D convolution
*   The output is then fed to 2 different dilated 1D convolution layers with sigmoid and tanh activations
*   The element-wise multiplication of 2 different activation values results in a skip connection
*   And the element-wise addition of a skip connection and output of causal 1D results in the residual



In [0]:
# music 21 library
from music21 import *

In [0]:
# reading midi file 
def read_midi(file):
  print("Loading music file",file)
  notes=[]
  notes_to_parse=None
  midi = converter.parse(file)
  s2 = instrument.partitionByInstrument(midi)
  for part in s2.parts:
    
  